In [ ]:
import pandas as pd
import webbrowser
import pyautogui
import time
import pyperclip
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog
from datetime import datetime
import locale
import unicodedata


def abrir_whatsapp_web():
    webbrowser.open("https://web.whatsapp.com")

def enviar_mensagens_nps(df, tipo_envio):
    root = tk.Tk()
    root.withdraw()
    
    mensagem_introducao = simpledialog.askstring("Mensagem de Introdução", "Digite a mensagem de introdução (ou deixe em branco para não enviar):")
    
    primeira_vez = True

    locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')
    mes_atual = datetime.now().strftime("%B").capitalize().replace("Março", "Marco")
    mes_anterior = (datetime.now().replace(day=1) - pd.DateOffset(days=1)).strftime("%B")
    
    mes_referencia = mes_atual if tipo_envio == "Parcial" else mes_anterior
    
    for index, row in df.iterrows():
        numero = row['Número']
        nome_loja = row['Nome Cliente']
        nps_score = row['NPS Score']
        aderencia_nps = row['% AderênciaNPS']
        doc_pendente = row['DOC. PENDENTES']
        
        if nps_score == "Sem Nota" or aderencia_nps == "Sem Nota" or pd.isna(numero):
            continue
        
        nps_score = nps_score * 100
        aderencia_nps = aderencia_nps * 100

        if tipo_envio == "Consolidado":
            if nps_score < 82:
                mensagem = (
                    f"📢 Resultados de {mes_referencia} - LOJA {nome_loja}\n"
                    f"⚠️ NPS: {nps_score:.2f}%\n"
                    f"📊 Aderência NPS: {aderencia_nps:.2f}%\n"
                    f"📄 Documentação Recusada: {doc_pendente}\n"
                    f"💬 Observações:\n"
                    f"Boa tarde! Notamos que o NPS ficou abaixo da meta de 82% no último mês. Sabemos que desafios acontecem, mas também sabemos do seu potencial para reverter esse cenário! 💪\n"
                    f"Que tal analisar os feedbacks recentes para identificar oportunidades de melhoria? Um detalhe que faz diferença é sempre reforçar com o cliente, ao final do serviço, que ele receberá uma pesquisa de satisfação. Isso aumenta a consciência sobre a importância da avaliação e pode contribuir para um resultado mais positivo."
                )
            else:
                mensagem = (
                    f"📢 Resultados de {mes_referencia} - LOJA {nome_loja}\n"
                    f"✅ NPS: {nps_score:.2f}%\n"
                    f"📊 Aderência NPS: {aderencia_nps:.2f}%\n"
                    f"📄 Documentação Recusada: {doc_pendente}\n"
                    f"💬 Observações:\n"
                    f"Bom dia! 🎉 Parabéns pelo resultado do último mês! O NPS reflete o compromisso da loja em entregar um atendimento de qualidade e superar as expectativas dos clientes. Parabéns pelo resultado alcançado! 👏"
                )
        else:
            if nps_score < 82:
                mensagem = (
                    f"📢 Resultados Parciais de {mes_referencia} - LOJA {nome_loja}\n"
                    f"⚠️ NPS ABAIXO DA META\n"
                    f"📊 NPS: {nps_score:.2f}%\n"
                    f"📉 Aderência NPS: {aderencia_nps:.2f}%\n"
                    f"📄 Documentação Recusada: {doc_pendente}\n"
                    f"💬 Observações:\n"
                    f"Boa tarde! Ainda temos tempo para reverter esse cenário e alcançar um resultado melhor neste mês. 💪 Agora é a hora de identificar oportunidades de melhoria e agir!"
                )
            else:
                mensagem = (
                    f"📢 Resultados Parciais de {mes_referencia} - LOJA {nome_loja}\n"
                    f"✅ NPS DENTRO DA META\n"
                    f"📊 NPS: {nps_score:.2f}%\n"
                    f"📈 Aderência NPS: {aderencia_nps:.2f}%\n"
                    f"📄 Documentação Recusada: {doc_pendente}\n"
                    f"💬 Observações:\n"
                    f"Bom dia! 🎉 Parabéns pelo ótimo desempenho até aqui! Seu compromisso com um atendimento de qualidade tem garantido esse resultado positivo. 👏"
                )
        
        if primeira_vez:
            abrir_whatsapp_web()
            primeira_vez = False
            time.sleep(7)

        try:
            pyautogui.hotkey('ctrl', 'alt', 'n')
            time.sleep(0.5)
            pyautogui.typewrite(numero.replace('+', ''))
            time.sleep(0.5)
            pyautogui.press('enter')
            time.sleep(0.5)
            
            if mensagem_introducao:
                pyperclip.copy(mensagem_introducao)
                pyautogui.hotkey('ctrl', 'v')
                pyautogui.press("enter")

            pyperclip.copy(mensagem)
            pyautogui.hotkey('ctrl', 'v')
            pyautogui.press("enter")
            print(f"Mensagem enviada para {numero}")
            time.sleep(0.5)

        except Exception as e:
            print(f"Erro ao enviar mensagem para {numero}: {e}")

    messagebox.showinfo("Concluído", "Envio de mensagens concluído com sucesso!")

def main():
    root = tk.Tk()
    root.withdraw()
    
    base_path = filedialog.askopenfilename(
        title="Selecione o arquivo do Excel",
        filetypes=[("Arquivos do Excel", "*.xlsx *.xls")]
    )
    
    if not base_path:
        messagebox.showinfo("Nenhum arquivo selecionado", "Nenhum arquivo foi selecionado. O programa será encerrado.")
        return
    
    df = pd.read_excel(base_path, sheet_name='ACOMPANHAMENTO', header=8)
    tipo_envio = simpledialog.askstring("Tipo de Envio", "Deseja enviar 'Parcial' (mês atual) ou 'Consolidado' (mês anterior)?")
    if tipo_envio not in ["Parcial", "Consolidado"]:
        messagebox.showerror("Erro", "Opção inválida. Encerrando programa.")
        return
    enviar_mensagens_nps(df, tipo_envio)

if __name__ == '__main__':
    main()